In [0]:
from pyspark.sql.functions import col
file_location = "/Workspace/Users/pawel.rubach@sgh.waw.pl/data/telco.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

from pyspark.sql.functions import trim, when 

#df_tc_dbl = df.withColumn('TotalCharges', col('TotalCharges').cast('double'))

#display(df.select(trim(col("TotalCharges"))))
#.cast('double')
df_tc_dbl = df.withColumn('TotalCharges', trim(col("TotalCharges")))

#df_tc_dbl['TotalCharges Trimmed'] = df_tc_dbl['TotalCharges Trimmed'].apply(lambda x: float(x) if x != '' else 0)
#df_tc_dbl.replace()
#df_tc_dbl['TotalCharges Trimmed'] = df_tc_dbl['TotalCharges Trimmed'].replace('', 0)

df_tc_dbl = df.withColumn("TotalCharges", \
       when(col('TotalCharges')==" " , None) \
          .otherwise(col('TotalCharges')))
df_tc_dbl = df_tc_dbl.withColumn('TotalCharges', col('TotalCharges').cast('double'))

#df_tc_dbl = df.withColumn('TotalCharges', df.select(trim(col("TotalCharges"))).cast('double'))
display(df_tc_dbl)

In [0]:
df_grp_contract_phone = df_tc_dbl.groupBy('Contract', 'PhoneService').count().sort(['Contract', 'PhoneService'])
#df_grp_contract_phone.show()
display(df_grp_contract_phone)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import round, avg, count
# Aggregate: Contract, PhoneService, Avg MonthlyCharges (rounded), count (rows)
df_agg = df_tc_dbl.groupBy('Contract', 'PhoneService') \
    .agg(round(avg('MonthlyCharges'), 2).alias('Avg MonthlyCharges'), count('Contract').alias('Count'))
display(df_agg)


In [0]:
type(df_agg.collect())
#df_agg.collect()
my_res = None
for row in df_agg.collect():
  #print(row)
  row_dict = row.asDict()
  #print(type(row_dict))
  if row_dict['Contract'] == 'One year' and row_dict['PhoneService'] == 'Yes':
    my_res = row_dict['Avg MonthlyCharges']
    print(type(my_res))
    print(my_res)